In [1]:
!pip install nltk
!pip install openpyxl

In [2]:
import pandas as pd
import re
import nltk
from nltk.util import ngrams
from collections import Counter
nltk.download('punkt')

[nltk_data] Downloading package punkt to /home/codespace/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [3]:
df=pd.read_excel('/workspaces/BMIG60303-assignments/All_Articles_Excel_Dec2019July2020.xlsx')

In [4]:
df.head()

,Date Added,Author,Title,Abstract,Year,Journal/Publisher,Volume,Issue,Pages,Accession Number,...,Unnamed: 84,Unnamed: 85,Unnamed: 86,Unnamed: 87,Unnamed: 88,Unnamed: 89,Unnamed: 90,Unnamed: 91,Unnamed: 92,Unnamed: 93
0,2020-07-31,"Zwerling, A",Understanding spending trends for tuberculosis,Total out-of-pocket spending decreased over th...,2020,Lancet Infectious Diseases,20,8,879-880,2428388309,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2020-07-31,"Zucoloto, GM, Pedro; Porto, Patricia",A propriedade industrial pode limitar o combat...,Esta nota técnica apresenta alguns dos pontos ...,2020,NaN,NaN,Issue,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2020-07-31,"Zimmer, MAZ, Anne K.; Weißer, Christian W.; Vo...",Hypernatremia—A Manifestation of COVID-19: A C...,We report for the first time therapy-resistant...,2020,A&A Practice,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2020-07-31,"Zignale, M",The experienced space between mobility and Cov...,If it is true that lived space represents our ...,2020,Documenti Geografici,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2020-07-31,"Zhu, XC, Bodong; Avadhanam, Rukmini Manasa; Sh...",Reading and connecting: using social annotatio...,Purpose - The COM-19 pandemic has forced many ...,2020,Information and Learning Sciences,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
df.columns

Index(['Date Added', 'Author', 'Title', 'Abstract', 'Year',
       'Journal/Publisher', 'Volume', 'Issue', 'Pages', 'Accession Number',
       'DOI', 'URL', 'Name of Database', 'Database Provider', 'Language',
       'Keywords', 'Unnamed: 16', 'Unnamed: 17', 'Unnamed: 18', 'Unnamed: 19',
       'Unnamed: 20', 'Unnamed: 21', 'Unnamed: 22', 'Unnamed: 23',
       'Unnamed: 24', 'Unnamed: 25', 'Unnamed: 26', 'Unnamed: 27',
       'Unnamed: 28', 'Unnamed: 29', 'Unnamed: 30', 'Unnamed: 31',
       'Unnamed: 32', 'Unnamed: 33', 'Unnamed: 34', 'Unnamed: 35',
       'Unnamed: 36', 'Unnamed: 37', 'Unnamed: 38', 'Unnamed: 39',
       'Unnamed: 40', 'Unnamed: 41', 'Unnamed: 42', 'Unnamed: 43',
       'Unnamed: 44', 'Unnamed: 45', 'Unnamed: 46', 'Unnamed: 47',
       'Unnamed: 48', 'Unnamed: 49', 'Unnamed: 50', 'Unnamed: 51',
       'Unnamed: 52', 'Unnamed: 53', 'Unnamed: 54', 'Unnamed: 55',
       'Unnamed: 56', 'Unnamed: 57', 'Unnamed: 58', 'Unnamed: 59',
       'Unnamed: 60', 'Unnamed: 61', 'Unn

In [20]:
df = df[['Date Added', 'Author', 'Title', 'Abstract', 'Year']]
df.dropna(subset=['Abstract'], how='all', inplace=True)
df['Abstract'] = df['Abstract'].str.lower()

In [21]:
df.head()

,Date Added,Author,Title,Abstract,Year
0,2020-07-31,"Zwerling, A",Understanding spending trends for tuberculosis,total out-of-pocket spending decreased over th...,2020
1,2020-07-31,"Zucoloto, GM, Pedro; Porto, Patricia",A propriedade industrial pode limitar o combat...,esta nota técnica apresenta alguns dos pontos ...,2020
2,2020-07-31,"Zimmer, MAZ, Anne K.; Weißer, Christian W.; Vo...",Hypernatremia—A Manifestation of COVID-19: A C...,we report for the first time therapy-resistant...,2020
3,2020-07-31,"Zignale, M",The experienced space between mobility and Cov...,if it is true that lived space represents our ...,2020
4,2020-07-31,"Zhu, XC, Bodong; Avadhanam, Rukmini Manasa; Sh...",Reading and connecting: using social annotatio...,purpose - the com-19 pandemic has forced many ...,2020


In [22]:
df.iloc[0]['Abstract']

"total out-of-pocket spending decreased over the same period; however, although the authors captured direct out-of-pocket spending on medical expenses, they did not include non-medical costs including loss of income, transport, and indirect economic costs due to tuberculosis (many of which are now being collected through who patient cost surveys) in their analysis. the authors' findings show that three countries with strong private sectors—democratic republic of the congo, nigeria, and pakistan—have out-of-pocket medical expenses as the primary source of tuberculosis spending. [...]trends over time and across countries can be used to monitor fluctuations in total tuberculosis spending and assess needs across regions."

In [23]:

from sklearn.feature_extraction.text import TfidfVectorizer

In [24]:
def part01(df):
    vectorizer = TfidfVectorizer()
    tfidf_matrix = vectorizer.fit_transform(df['Abstract'])
    tfidf_df = pd.DataFrame(tfidf_matrix.toarray(), columns=vectorizer.get_feature_names_out())

    return tfidf_df

In [25]:
df_tfidf = part01(df.sample(frac=0.1, random_state=42))

ValueError: np.nan is an invalid document, expected byte or unicode string.

In [26]:
df_tfidf.head()

,00,000,0000,00001,000022583,0000471,00006,00007,0001,00015,...,黄芩可通过多途径,黄芩改善covid,黄芩核心靶标,黄芩素,黄芩药对具有抗炎,黄芩药对改善covid,黄芩药对改善新型冠状病毒,黄芩药对的临床应用提供参考,黄芪,鼻塞
0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.029039,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [13]:
import pandas as pd

def part02(df_vectors):
    top_words_per_abstract = []
    for i in range(min(10, df_vectors.shape[0])):
        doc_vector = df_vectors.iloc[i]
        top_words = doc_vector.sort_values(ascending=False).head(5)
        top_words_list = [(word, score) for word, score in top_words.items()]
        top_words_per_abstract.append(top_words_list)

    return top_words_per_abstract


In [28]:
top_words = part02(df_tfidf)

In [29]:
top_words

[[('本文分析武汉某新建治疗新型冠状病毒肺炎医院院所配备的药品目录和四味专有中药组方', 0.3779644730092272),
  ('提高临床治疗效果', 0.3779644730092272),
  ('分布', 0.3779644730092272),
  ('代谢和排泄四个方面总结了中西药相互作用的机制和用药监护要点', 0.3779644730092272),
  ('提高用药安全性具有重要意义', 0.3779644730092272)],
 [('deprivation', 0.41341520534712756),
  ('chennai', 0.351177686586386),
  ('megacity', 0.30728047576308776),
  ('nbr', 0.27614158078264833),
  ('wards', 0.2150532548816092)],
 [('coefficients', 0.2721853810972516),
  ('data', 0.2179481927069262),
  ('the', 0.1984725447774574),
  ('set', 0.17672111264349175),
  ('fit', 0.17585399792423398)],
 [('the', 0.2494100249300536),
  ('technology', 0.23192688034845169),
  ('synchronicity', 0.21967073876524978),
  ('auspicious', 0.21967073876524978),
  ('1950s', 0.21967073876524978)],
 [('同时', 0.2901346225257028),
  ('cov', 0.23550810382798631),
  ('的来源', 0.15750774213227178),
  ('没有证据显示宠物会感染新型冠状病毒', 0.15750774213227178),
  ('彰显了宠物与人的亲密程度及其家庭地位之高', 0.15750774213227178)],
 [('resection', 0.26304733570668354),
  ('surgi

In [27]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer

def part03(df_vectors, query_string):
    vectorizer = TfidfVectorizer(vocabulary=df_vectors.columns)
    query_vector = vectorizer.fit_transform([query_string])
    cosine_similarities = cosine_similarity(query_vector, df_vectors).flatten()
    top_5_indices = cosine_similarities.argsort()[-5:][::-1]

    return top_5_indices


In [28]:
query_string = "machine learning"
op_docs = part03(df_tfidf, query_string)

In [29]:
op_docs

array([4106, 4354, 3413, 3113, 4658])

In [30]:
df.iloc[[4106, 4354, 3413, 3113, 4658]]['Abstract']

6092    over the past years, several zoonotic viruses ...
6420    coronavirus disease 2019, i.e. covid-19, start...
5081    two recent lancet and lancet oncology papers r...
4625    whether weather plays a part in the transmissi...
6737    武 汉 新 型 冠 状 病 毒 肺 炎 是 指 由 新 型 冠 状 病 毒 (2019-nc...
Name: Abstract, dtype: object

# Part 1

## Test Regex

In [9]:
pattern = r'\bItaly|Italian\b'
re.findall(pattern, 'Italy ITALY ITALIAN Italian italy italian ItaLian iTalY', re.IGNORECASE)

['Italy',
 'ITALY',
 'ITALIAN',
 'Italian',
 'italy',
 'italian',
 'ItaLian',
 'iTalY']

In [10]:
matches_l = []
for index, row in df.iterrows():
    abstract = row['Abstract']
    if isinstance(abstract, str):
        matches = re.findall(pattern, abstract, re.IGNORECASE)
        match_count = len(matches)
        if match_count > 0:
            matches_l.append((matches, match_count, abstract))
top_matches = sorted(matches_l, key=lambda x: x[1], reverse=True)[:10]

In [11]:
for i in top_matches:
    print(i[0])
    print(i[1])
    print(i[2])
    

['Italy', 'Italy', 'Italy', 'Italian', 'Italy', 'Italy', 'Italy', 'Italy', 'Italy', 'Italian', 'Italy', 'Italy']
12
Italy is currently experiencing an epidemic of COVID-19 which emerged in the Lombardy region . During the interval between February 25-29, 2020, we identified 46 cases of COVID-19 reported in 21 countries in Europe, Africa, North America, and South America which were either in individuals with recent travel from Italy, or who had presumed infection by a traveler from Italy 2. In six cases, in four of the affected countries (Switzerland, France, Austria, Croatia), land travel was a likely route of introduction, or was documented to have been the route of introduction. We used air travel volume between Italian cities and cities in other countries as an index of connectedness, using data available from the International Air Transport Association (IATA) for February 2015 (2.61 million total departing international air passengers from Italy). We used the methods of Fraser et a

# Part 2
## Test tokenizer

In [12]:
from nltk.tokenize import wordpunct_tokenize
text = 'This is a Italy of the ITALY. ITALIAN Italian in the realm. That; right there is italy "italian" ItaLian iTalY.'
tokens = wordpunct_tokenize(text)
print(tokens)

['This', 'is', 'a', 'Italy', 'of', 'the', 'ITALY', '.', 'ITALIAN', 'Italian', 'in', 'the', 'realm', '.', 'That', ';', 'right', 'there', 'is', 'italy', '"', 'italian', '"', 'ItaLian', 'iTalY', '.']


In [13]:
bigrams = Counter()
pattern = re.compile(r'\b(Italy|Italian)\b', re.IGNORECASE)
for index, row in df.iterrows():
    abstract = row['Abstract']
    if isinstance(abstract, str):
        tokens = wordpunct_tokenize(abstract)
        bigrams_list = list(ngrams(tokens, 2))
        re_search = [bigram for bigram in bigrams_list if pattern.search(bigram[0])]
        bigrams.update(re_search)

In [14]:
common_bigrams = bigrams.most_common(20)
for i in common_bigrams:
    print(str(i[0])+"\t"+ str(i[1]))

('Italy', ',')	703
('Italy', '.')	282
('Italy', 'and')	274
('Italy', 'is')	94
('Italy', '(')	79
('Italian', 'regions')	77
('Italy', 'has')	69
('Italian', 'Society')	57
('Italy', 'was')	56
('Italy', ')')	54
('Italian', 'population')	34
('Italian', 'government')	33
('Italian', 'National')	28
('Italian', 'Ministry')	28
('Italy', '),')	27
('Italy', 'in')	26
('Italian', 'Government')	23
('Italian', 'COVID')	19
('Italy', 'on')	18
('Italy', 'with')	18


# Part 3

## Test tokenizer

In [15]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/codespace/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [16]:
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))

In [17]:
text = 'This is a Italy of the ITALY. ITALIAN Italian in the realm. That; right there is italy "italian" ItaLian iTalY.'
tokens = wordpunct_tokenize(text)
cleaned_tokens = [token for token in tokens if token.isalpha() and token.lower() not in stop_words]
cleaned_tokens

['Italy',
 'ITALY',
 'ITALIAN',
 'Italian',
 'realm',
 'right',
 'italy',
 'italian',
 'ItaLian',
 'iTalY']

In [18]:
bigrams = Counter()
pattern = re.compile(r'\b(Italy|Italian)\b', re.IGNORECASE)
for index, row in df.iterrows():
    abstract = row['Abstract']
    if isinstance(abstract, str):
        tokens = wordpunct_tokenize(abstract)
        cleaned_tokens = [token for token in tokens if token.isalnum() and token.lower() not in stop_words]
        bigrams_list = list(ngrams(cleaned_tokens, 2))
        re_search = [bigram for bigram in bigrams_list if pattern.search(bigram[0])]
        bigrams.update(re_search)

In [19]:
common_bigrams = bigrams.most_common(20)
common_bigrams

[(('Italy', 'Spain'), 177),
 (('Italian', 'regions'), 77),
 (('Italian', 'Society'), 57),
 (('Italy', 'one'), 56),
 (('Italy', 'first'), 52),
 (('Italy', 'Iran'), 49),
 (('Italian', 'population'), 34),
 (('Italian', 'government'), 33),
 (('Italy', 'Germany'), 29),
 (('Italy', 'France'), 29),
 (('Italian', 'National'), 28),
 (('Italy', 'China'), 28),
 (('Italian', 'Ministry'), 28),
 (('Italy', 'South'), 26),
 (('Italian', 'Government'), 23),
 (('Italy', 'USA'), 23),
 (('Italy', 'March'), 22),
 (('Italy', 'United'), 20),
 (('Italian', 'COVID'), 19),
 (('Italian', 'provinces'), 17)]

In [3]:
df = pd.read_csv('1557tweets.csv')

In [4]:
df.head()

,text
0,Kodak Black - Codeine Dreaming &gt;&gt;
1,"give me some morphine, is there any more to do?"
2,"just like nicotine,\nheroin, morphine"
3,Codeine hella calling my name 🙄
4,Minus how much coke you youngins be on. Like y...


In [6]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     /home/codespace/nltk_data...


True

In [7]:
import nltk
from nltk.corpus import wordnet as wn
nltk.download('wordnet')

def part01():
    synset_consume = wn.synset('consume.v.02')
    hyponyms = synset_consume.closure(lambda s: s.hyponyms())
    return set(hyponyms)
result = part01()
print(result)
print(len(result))


{Synset('gluttonize.v.01'), Synset('sip.v.01'), Synset('receive.v.12'), Synset('pub-crawl.v.01'), Synset('tuck_in.v.01'), Synset('hit.v.15'), Synset('wash_down.v.01'), Synset('ruminate.v.01'), Synset('pop.v.11'), Synset('guzzle.v.01'), Synset('fare.v.02'), Synset('dunk.v.03'), Synset('wine_and_dine.v.01'), Synset('eat_up.v.01'), Synset('sup.v.01'), Synset('eat_out.v.01'), Synset('toss_off.v.02'), Synset('crop.v.05'), Synset('eat.v.01'), Synset('raven.v.02'), Synset('lunch.v.01'), Synset('inject.v.04'), Synset('drop.v.17'), Synset('guggle.v.03'), Synset('eat.v.02'), Synset('tank.v.02'), Synset('use.v.02'), Synset('mess.v.01'), Synset('chain-smoke.v.01'), Synset('port.v.07'), Synset('slurp.v.01'), Synset('nibble.v.03'), Synset('fill_up.v.04'), Synset('mainline.v.01'), Synset('degust.v.01'), Synset('partake.v.03'), Synset('pitch_in.v.01'), Synset('board.v.03'), Synset('break_bread.v.01'), Synset('carry.v.33'), Synset('take_out.v.12'), Synset('swill.v.02'), Synset('picnic.v.01'), Synset('s

In [9]:
def part02(synset_set):
    lemmas = {lemma for synset in synset_set for lemma in synset.lemma_names()}
    filtered_lemmas = {lemma for lemma in lemmas if '_' not in lemma}
    return filtered_lemmas

synsets = part01()
lemmas = part02(synsets)
print(lemmas)
print(len(lemmas))


{'bolt', 'gulp', 'souse', 'tope', 'fress', 'pall', 'guzzle', 'dunk', 'snack', 'whiff', 'pick', 'gurgle', 'inject', 'gobble', 'hit', 'englut', 'guggle', 'feast', 'drop', 'soak', 'dip', 'finish', 'swill', 'partake', 'forage', 'hold', 'stuff', 'victual', 'gormandise', 'breakfast', 'gluttonise', 'use', 'tipple', 'overeat', 'inhale', 'wine', 'gorge', 'dope', 'satiate', 'ruminate', 'port', 'habituate', 'suckle', 'ingurgitate', 'gluttonize', 'gourmandize', 'lunch', 'claret', 'lap', 'dine', 'drink', 'degust', 'cannibalise', 'inebriate', 'feed', 'imbibe', 'smoke', 'cannibalize', 'carry', 'sample', 'sup', 'board', 'graze', 'kill', 'browse', 'prey', 'drug', 'junket', 'nosh', 'swallow', 'slurp', 'piece', 'fuddle', 'pig', 'quaff', 'overindulge', 'brunch', 'suck', 'predate', 'free-base', 'binge', 'base', 'peck', 'eat', 'gormandize', 'banquet', 'chain-smoke', 'sip', 'engorge', 'fare', 'down', 'try', 'receive', 'cloy', 'mess', 'crop', 'picnic', 'glut', 'pub-crawl', 'overgorge', 'wolf', 'snort', 'tank'

In [12]:
!pip install spacy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.7/31.7 MB 90.4 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 40.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 865.0/865.0 kB 17.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.0/18.0 MB 79.5 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 85.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 66.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 25.2 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.1.0
    Uninstalling numpy-2.1.0:
      Successfully uninstalled numpy-2.1.0


In [23]:
import spacy
from nltk.corpus import stopwords
from collections import Counter
import string

import en_core_web_sm
nlp = en_core_web_sm.load()
def part03(texts, consume_terms):
    stop_words = set(stopwords.words('english'))
    punctuation = set(string.punctuation)
    bigram_counter = Counter()
    for text in texts:
        doc = nlp(text)
        tokens = [
            token.lemma_.lower() for token in doc
            if token.lemma_.lower() not in stop_words and token.lemma_ not in punctuation and not token.is_punct
        ]
        for i in range(len(tokens) - 1):
            first_word = tokens[i]
            second_word = tokens[i + 1]
            if first_word in consume_terms:
                bigram_counter[(first_word, second_word)] += 1
    return bigram_counter

# Example usage:
# Load the tweets from the CSV file
df = pd.read_csv('1157tweets.csv')

# Get the list of consume terms by calling part01 and part02
consume_terms = part02(part01())

# Call part03 to process the tweets and find relevant bigrams
res = part03(df['text'], consume_terms)

# Print the most common bigrams
print(res.most_common(10))


ValueError: numpy.dtype size changed, may indicate binary incompatibility. Expected 96 from C header, got 88 from PyObject

In [17]:
! --yes -r
! --yes -r


Usage:   
  pip uninstall [options] <package> ...
  pip uninstall [options] -r <requirements file> ...

-r option requires 1 argument

Usage:   
  pip install [options] <requirement specifier> [package-index-options] ...
  pip install [options] -r <requirements file> [package-index-options] ...
  pip install [options] [-e] <vcs project url> ...
  pip install [options] [-e] <local project path> ...
  pip install [options] <archive url/path> ...

no such option: --yes


In [24]:
try:
    nlp = spacy.load("en_core_web_sm")
except OSError:
    from spacy.cli import download
    download("en_core_web_sm")
    nlp = spacy.load("en_core_web_sm")

NameError: name 'spacy' is not defined

In [25]:
import spacy

ValueError: numpy.dtype size changed, may indicate binary incompatibility. Expected 96 from C header, got 88 from PyObject